# Use Langchain to query a localai model

## Install the required packages to the python environment (Tagged to 0.28.1 because openai release version 1 yesterday and broke langchain.  Langchain pull request pending to resolve)

In [19]:
%pip install python-dotenv
%pip install requests
%pip install langchain
%pip install pip install pyodbc
%pip install --upgrade openai==0.28.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 68.7/68.7 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE') # This will tell the openai library where to send requests
print(OPENAI_API_BASE)

None


## List available local models

In [ ]:
import requests

response = requests.get(f'{OPENAI_API_BASE}/models')
print(response.json())

{'object': 'list', 'data': [{'id': 'gpt4all-j', 'object': 'model'}, {'id': 'ggml-gpt4all-j', 'object': 'model'}]}


## Connect Langchain to localai instance using the desired model

In [5]:
from langchain.llms import OpenAI

# for local
llm = OpenAI(model_name='gpt4all-j')
llm("Are you working?")

# for chatgpt
# from langchain.chat_models import ChatOpenAI
# llm = OpenAI()
# chat_model = ChatOpenAI()

# Answer question
# llm.predict("hi!")

# Chat
# chat_model.predict("hi!")


'\n\nGood morning\n\nGood morning! How are you?'

## Create a database for the AI to query

First get a local database running with docker
```
docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=Password123" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2022-latest
```

Next create connect to SQL and import or create a database

Create a READ ONLY user for connecting to the database.  This is important because it's possible the AI could decide to do insert or delete statements.  We don't want that.

Here is the example SQL database for this test
```
CREATE DATABASE TestDb

CREATE TABLE People (
    ID INT PRIMARY KEY,
    FirstName NVARCHAR(50),
    LastName NVARCHAR(50),
    Age INT
)

INSERT INTO People (ID, FirstName, LastName, Age)
VALUES (1, 'John', 'Doe', 30)

INSERT INTO People (ID, FirstName, LastName, Age)
VALUES (2, 'Johny', 'Does', 60)

INSERT INTO People (ID, FirstName, LastName, Age)
VALUES (3, 'Jane', 'Doe', 20)
```

## Connect to the database
use sql_database toolkit https://python.langchain.com/docs/integrations/toolkits/sql_database

In [6]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

server = '127.0.0.1'
database = 'TestDb'
username = 'sa'
password = 'Password123'
driver = 'ODBC Driver 17 for SQL Server'

conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"

# Only include the People table
db = SQLDatabase.from_uri(conn_str, include_tables=['People'])

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)

## Query the database using the agent_executor

In [8]:
agent_executor.run("Who is the oldest person in the database?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: People
Thought: I should query the schema of the People table to see what fields are available.
Action: sql_db_schema
Action Input: People
Observation: 
CREATE TABLE [People] (
	[ID] INTEGER NOT NULL, 
	[FirstName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[LastName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Age] INTEGER NULL, 
	CONSTRAINT [PK__People__3214EC273DC4372C] PRIMARY KEY ([ID])
)

/*
3 rows from People table:
ID	FirstName	LastName	Age
1	John	Doe	30
2	Johny	Does	60
3	Jane	Doe	20
*/
Thought: The relevant field here is Age, so I should query the database for the oldest person by Age.
Action: sql_db_query
Action Input: SELECT TOP 10 FirstName, LastName, Age FROM People ORDER BY Age DESC
Observation: [('Johny', 'Does', 60), ('John', 'Doe', 30), ('Jane', 'Doe', 20)]
Thought: I now know the final answer. 
Final Answer: The oldest person in the database is 

'The oldest person in the database is Johny Does, aged 60.'